# **Frozen Lake (Q-Learning)**
Frozen Lake adalah salah satu lingkungan dalam pustaka OpenAI Gym yang sering digunakan untuk mengajarkan dan menguji algoritma reinforcement learning. Di sini, agen harus belajar untuk mencapai tujuan di danau beku tanpa jatuh ke dalam lubang es.

## **Import Modules dan Packages**


Untuk mengimplementasikan algoritma Q-Learning pada lingkungan Frozen Lake, kita memerlukan beberapa modul dan paket dari Python. Berikut ini adalah daftar modul dan paket yang perlu diimpor serta penjelasan singkat mengenai kegunaannya:

1. NumPy: Untuk operasi matematika dan manajemen array.
2. Gym (OpenAI Gym): Untuk simulasi lingkungan reinforcement learning seperti Frozen Lake.

Pertama, pastikan Anda telah menginstal paket `gym` dan `numpy`. Jika belum, Anda bisa menginstalnya menggunakan `pip`:

In [1]:
!pip install gym==0.17.3 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.5.1 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 8.12.3 which is incompatible.
spyder-kernels 2.5.0 requires ipython!=8.17.1,<9,>=8.13.0; python_version > "3.8", but you have ipython 8.12.3 which is incompatible.


Selanjutnya mengimpor modul dan paket yang dibutuhkan.

In [1]:
import numpy as np
import gym
import pickle

## **Inisialisasi Environment**

### Peta Frozen Lake

Frozen Lake adalah sebuah permainan grid di mana seorang agen harus bergerak dari posisi awal (start, `S`) ke posisi tujuan (goal, `G`) melalui petak-petak es yang aman (frozen, `F`) sambil menghindari lubang (hole, `H`). Peta ini diwakili dalam bentuk string dua dimensi, di mana setiap elemen string adalah sebuah baris dalam grid.

### Struktur Peta

Mari kita bahas setiap karakter dalam peta:
- **S**: Start, posisi awal agen.
- **F**: Frozen, petak es yang aman untuk dilewati.
- **H**: Hole, lubang yang menyebabkan agen jatuh dan episode berakhir.
- **G**: Goal, tujuan akhir yang harus dicapai oleh agen.

Berikut adalah daftar peta yang akan dibuat:

```python
peta = [
    ['SFFF', 'FHFH', 'FFFH', 'HFFG'],
    ['SFFF', 'FFHF', 'HFFF', 'HFFG'],
    ['SHFF', 'FHFH', 'FFFH', 'HHFG'],
    ['SFFF', 'HHFF', 'FFFF', 'HFFG'],
    ['SFFH', 'FFFH', 'HFFH', 'HHFG']
]
```

### Penjelasan Setiap Peta

Setiap elemen dalam daftar `peta` adalah sebuah peta dengan ukuran 4x4. Mari kita lihat masing-masing peta secara detail:

1. **Peta 1:**
   ```
   S F F F
   F H F H
   F F F H
   H F F G
   ```
   - `S` (Start) di kiri atas (0, 0).
   - `G` (Goal) di kanan bawah (3, 3).
   - Lubang (`H`) tersebar di beberapa tempat, seperti (1, 1), (1, 3), (2, 3), dan (3, 0).

2. **Peta 2:**
   ```
   S F F F
   F F H F
   H F F F
   H F F G
   ```
   - `S` di kiri atas (0, 0).
   - `G` di kanan bawah (3, 3).
   - Lubang (`H`) di (2, 0), (1, 2), dan (3, 0).

3. **Peta 3:**
   ```
   S H F F
   F H F H
   F F F H
   H H F G
   ```
   - `S` di kiri atas (0, 0).
   - `G` di kanan bawah (3, 3).
   - Lubang (`H`) di (0, 1), (1, 1), (1, 3), (2, 3), (3, 0), dan (3, 1).

4. **Peta 4:**
   ```
   S F F F
   H H F F
   F F F F
   H F F G
   ```
   - `S` di kiri atas (0, 0).
   - `G` di kanan bawah (3, 3).
   - Lubang (`H`) di (1, 0), (1, 1), dan (3, 0).

5. **Peta 5:**
   ```
   S F F H
   F F F H
   H F F H
   H H F G
   ```
   - `S` di kiri atas (0, 0).
   - `G` di kanan bawah (3, 3).
   - Lubang (`H`) di (0, 3), (1, 3), (2, 0), (2, 3), (3, 0), dan (3, 1).

In [2]:
# Membuat daftar peta
peta = [
    ['SFFF','FHFH','FFFH','HFFG'],
    ['SFFF','FFHF','HFFF','HFFG'],
    ['SHFF','FHFH','FFFH','HHFG'],
    ['SFFF','HHFF','FFFF','HFFG'],
    ['SFFH','FFFH','HFFH','HHFG']
]

Baris kode `env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta[0])` membuat lingkungan `FrozenLake-v0` dengan menggunakan peta pertama dari daftar peta yang disediakan (`peta[0]`). Dengan `is_slippery=False`, permukaan danau tidak licin, membuat pergerakan agen menjadi deterministik. Variabel `env` menyimpan instance dari lingkungan ini untuk melatih agen dengan algoritma pembelajaran penguatan seperti Q-Learning.

In [3]:
# Load Environment
env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[0])

Baris kode di bawah digunakan untuk menghitung berapa banyak petak atau posisi yang tersedia di atas permukaan es (state) dan berapa banyak langkah yang dapat diambil oleh agen untuk bergerak di sekitar permukaan es (action).

In [4]:
n_observations = env.observation_space.n
n_actions      = env.action_space.n

print('Banyak State  : ' + str(n_observations))
print('Banyak Action : ' + str(n_actions))

Banyak State  : 16
Banyak Action : 4


Pertama, `n_observations = env.observation_space.n` mengambil jumlah petak di atas permukaan es, sementara `n_actions = env.action_space.n` mengambil jumlah langkah yang bisa diambil oleh agen, seperti maju, mundur, atau ke samping.

Kemudian, kedua pernyataan `print` digunakan untuk mencetak jumlah petak dan jumlah langkah ke layar agar kita dapat melihat berapa banyak petak dan langkah yang ada dalam permainan tersebut.

- Hasil "Banyak State: 16" berarti terdapat total 16 petak atau posisi yang dapat diakses oleh agen di atas permukaan es dalam permainan Frozen Lake. Masing-masing petak mewakili satu keadaan (state) yang dapat diamati oleh agen
- Hasil "Banyak Action: 4" berarti agen memiliki empat aksi yang dapat diambil dalam setiap keadaan. Dalam permainan Frozen Lake, agen dapat memilih antara empat aksi: "maju" (up), "mundur" (down), "ke kiri" (left), dan "ke kanan" (right).

In [5]:
ACTION = ["KIRI","BAWAH","KANAN","ATAS"]

Variabel `ACTION` adalah daftar yang berisi empat string: "KIRI", "BAWAH", "KANAN", dan "ATAS". Setiap string mewakili salah satu dari empat aksi yang dapat diambil oleh agen dalam permainan Frozen Lake. Berikut adalah makna dari setiap aksi:

- "KIRI": Agen bergerak ke kiri.
- "BAWAH": Agen bergerak ke bawah.
- "KANAN": Agen bergerak ke kanan.
- "ATAS": Agen bergerak ke atas.

Dengan menggunakan daftar ini, kita dapat merujuk ke aksi yang diambil oleh agen dengan menggunakan indeks dalam daftar. Misalnya, untuk mengambil aksi "KIRI", kita akan menggunakan `ACTION[0]`, untuk "BAWAH" akan menggunakan `ACTION[1]`, dan seterusnya.

In [6]:
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


Perintah `env.reset()` digunakan untuk mengatur ulang lingkungan ke keadaan awalnya, yang berarti mengatur ulang agen ke posisi awalnya di permukaan es. Ini berguna untuk memulai episode baru dalam permainan.

Sementara perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar. Ini memperlihatkan keadaan terbaru dari permainan, termasuk posisi agen, tujuan, petak es, dan lubang.

Ketika kedua perintah digabungkan, `env.reset()` akan mengatur ulang permainan ke keadaan awalnya dan `env.render()` akan menampilkan visualisasi permainan dalam keadaan tersebut.

In [7]:
# Langkah 1 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 1
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


Langkah pertama yang diambil adalah perintah `env.step(2)`, yang berarti agen mencoba untuk bergerak ke kanan. Kemudian, kode mencetak hasil langkah tersebut: posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`). Setelah itu, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah diambil.

In [8]:
# Langkah 2 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 2
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


Langkah kedua adalah agen mencoba bergerak ke kanan lagi dengan perintah `env.step(2)`. Hasil langkahnya, seperti posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`), dicetak. Setelah itu, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah kedua diambil.

In [9]:
# Langkah 3 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print('New State : {}'.format(new_state))
print('Reward    : {}'.format(reward))
print('Done      : {}'.format(done))

# Menampilkan visualisasi lingkungan
env.render()

New State : 6
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


Langkah ketiga adalah agen mencoba bergerak ke bawah dengan perintah `env.step(1)`. Hasil langkahnya dicetak, termasuk posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`). Kemudian, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah ketiga diambil.

In [10]:
# Langkah 4 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print('New State : {}'.format(new_state))
print('Reward    : {}'.format(reward))
print('Done      : {}'.format(done))

# Menampilkan visualisasi lingkungan
env.render()

New State : 10
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


Langkah keempat adalah agen mencoba bergerak ke bawah dengan perintah `env.step(1)`. Hasil langkahnya dicetak, termasuk posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`). Setelah itu, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah keempat diambil.

In [11]:
# Langkah 5 (ke Bawah)
new_state, reward, done, info = env.step(1)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 14
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


Langkah kelima adalah agen mencoba bergerak ke bawah dengan perintah `env.step(1)`. Hasil langkahnya dicetak, termasuk posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`). Kemudian, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah kelima diambil.

In [12]:
# Langkah 6 (ke Kanan)
new_state, reward, done, info = env.step(2)

# Menampilkan informasi
print(f"New State : {new_state}")
print(f"Reward    : {reward}")
print(f"Done      : {done}")

# Menampilkan visualisasi lingkungan
env.render()

New State : 15
Reward    : 1.0
Done      : True
  (Right)
SFFF
FHFH
FFFH
HFFG


Langkah keenam adalah agen mencoba bergerak ke kanan dengan perintah `env.step(2)`. Hasil langkahnya dicetak, termasuk posisi baru agen (`New State`), reward yang diterima (`Reward`), dan apakah episode permainan telah selesai (`Done`). Setelah itu, perintah `env.render()` digunakan untuk menampilkan visualisasi lingkungan permainan Frozen Lake ke layar dalam keadaan baru setelah langkah keenam diambil.

## **Melakukan Training Model atau Agent**

In [14]:
def train_agent(env, n_episodes=10000, max_iter_episode=100, exploration_proba=1, exploration_decreasing_decay=0.001, min_exploration_proba=0.01, gamma=0.99, lr=0.1):
    # Inisialisasi Q-table dengan ukuran berdasarkan jumlah state dan aksi
    Q_table = np.zeros((env.observation_space.n, env.action_space.n))

    # List untuk menyimpan reward dari setiap episode
    rewards_per_episode = []

    # Loop melalui setiap episode
    for episode in range(n_episodes):
        # Reset lingkungan untuk memulai episode baru dan mendapatkan state awal
        state = env.reset()

        # Inisialisasi total reward episode menjadi 0
        episode_reward = 0

        # Loop melalui setiap iterasi dalam episode
        for _ in range(max_iter_episode):
            # Pilih tindakan berdasarkan probabilitas eksplorasi atau menggunakan kebijakan Q
            if np.random.uniform(0, 1) < exploration_proba:
                action = env.action_space.sample()  # Aksi acak (eksplorasi)
            else:
                action = np.argmax(Q_table[state, :])  # Aksi terbaik berdasarkan Q-table (eksploitasi)

            # Ambil langkah berdasarkan tindakan yang dipilih
            next_state, reward, done, _ = env.step(action)

            # Update Q-value berdasarkan reward yang diterima dan perkiraan nilai Q di state berikutnya
            Q_table[state, action] = (1 - lr) * Q_table[state, action] + lr * (reward + gamma * np.max(Q_table[next_state, :]))

            # Tambahkan reward dari langkah ini ke total reward episode
            episode_reward += reward
            state = next_state  # Pindah ke state berikutnya

            # Hentikan episode jika mencapai terminal state
            if done:
                break

        # Kurangi probabilitas eksplorasi seiring berjalannya waktu
        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay * episode))

        # Simpan total reward episode ke dalam list
        rewards_per_episode.append(episode_reward)

    # Cetak rata-rata reward per 1000 episode
    print("Rata-Rata Reward per 1000 Episode")
    for i in range(10):
        print((i + 1) * 1000, " : Rata-Rata Reward: ", np.mean(rewards_per_episode[1000 * i:1000 * (i + 1)]))

    # Kembalikan Q-table yang telah dilatih
    return Q_table

Fungsi `train_agent` mengimplementasikan algoritma Q-learning untuk melatih agen dalam suatu lingkungan yang diberikan. Selama pelatihan, agen memperbarui tabel Q berdasarkan pengalaman interaksi dengan lingkungan, di mana setiap langkah yang diambil menghasilkan reward yang akan memengaruhi pembelajaran agen.

Probabilitas eksplorasi digunakan untuk memutuskan apakah agen akan melakukan aksi acak untuk eksplorasi atau memilih tindakan terbaik berdasarkan tabel Q saat ini untuk eksploitasi. Selama pelatihan, probabilitas eksplorasi secara bertahap dikurangi. Pada akhirnya, fungsi mengembalikan tabel Q yang telah dilatih, yang digunakan agen untuk membuat keputusan saat berinteraksi dengan lingkungan.

In [15]:
Q_table_all = []  # List kosong untuk menyimpan semua tabel Q dari setiap lingkungan

for peta_env in peta:  # Loop melalui setiap peta dalam daftar peta
    # Load Environment untuk setiap peta dengan konfigurasi tertentu
    env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta_env)
    env.reset()  # Reset lingkungan ke keadaan awal

    print('Peta : ')
    print(peta_env)  # Cetak peta yang sedang digunakan

    # Melatih Agent pada lingkungan saat ini
    Q_table = train_agent(env)  # Panggil fungsi train_agent untuk melatih agen di lingkungan saat ini

    # Menyimpan Q_table untuk lingkungan saat ini ke dalam list
    Q_table_all.append(Q_table)  # Tambahkan Q_table ke dalam list Q_table_all

    print()  # Cetak baris kosong untuk pemisah antara lingkungan yang berbeda

Peta : 
['SFFF', 'FHFH', 'FFFH', 'HFFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.25
2000  : Rata-Rata Reward:  0.735
3000  : Rata-Rata Reward:  0.912
4000  : Rata-Rata Reward:  0.967
5000  : Rata-Rata Reward:  0.986
6000  : Rata-Rata Reward:  0.995
7000  : Rata-Rata Reward:  0.989
8000  : Rata-Rata Reward:  0.992
9000  : Rata-Rata Reward:  0.99
10000  : Rata-Rata Reward:  0.992

Peta : 
['SFFF', 'FFHF', 'HFFF', 'HFFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.417
2000  : Rata-Rata Reward:  0.852
3000  : Rata-Rata Reward:  0.957
4000  : Rata-Rata Reward:  0.982
5000  : Rata-Rata Reward:  0.989
6000  : Rata-Rata Reward:  0.995
7000  : Rata-Rata Reward:  0.996
8000  : Rata-Rata Reward:  0.997
9000  : Rata-Rata Reward:  0.992
10000  : Rata-Rata Reward:  0.994

Peta : 
['SHFF', 'FHFH', 'FFFH', 'HHFG']
Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.097
2000  : Rata-Rata Reward:  0.612
3000  : Rata-Rata Reward:  0.848
4000  : Rata-R

Dalam loop ini, setiap lingkungan dari daftar `peta` dimuat, agen dilatih di lingkungan tersebut, dan tabel Q hasil latihan disimpan ke dalam list `Q_table_all` untuk digunakan nanti. Proses ini diulang untuk setiap lingkungan dalam `peta`.

## **Memainkan Agent yang Telah Dilatih**

Kode ini digunakan untuk memilih peta yang akan digunakan dalam lingkungan permainan Frozen Lake. Anda dapat mengubah nilai dari `index_peta` untuk memilih peta tertentu dari daftar `peta`. Lingkungan permainan kemudian dibuat menggunakan `gym.make()` dengan konfigurasi `FrozenLake-v0` dan deskripsi peta yang sesuai dengan pilihan Anda. Parameter `is_slippery=False` menetapkan perilaku permukaan es agar tetap stabil tanpa geseran, dan `env.reset()` digunakan untuk mengatur ulang lingkungan ke keadaan awal.

In [16]:
index_peta = 0 # silahkan pilih peta

env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[index_peta])
env.reset()

0

Loop `for` ini digunakan untuk mengikuti langkah-langkah agen dalam lingkungan Frozen Lake berdasarkan tabel Q yang telah dilatih. Pada setiap iterasi, agen memilih tindakan terbaik berdasarkan tabel Q yang telah dilatih untuk langkah sebelumnya atau langkah awal jika ini adalah langkah pertama. Agen kemudian melakukan langkah tersebut dalam lingkungan, mendapatkan informasi seperti state baru, reward yang diterima, dan apakah episode permainan telah selesai. Selanjutnya, informasi tersebut dicetak dan visualisasi lingkungan diperbarui. Proses ini diulang untuk total 6 langkah.

In [17]:
for langkah in range(1, 7):
    if langkah == 1:
        best_action = np.argmax(Q_table_all[index_peta][0])
    else:
        best_action = np.argmax(Q_table_all[index_peta][current_state])

    new_state, reward, done, info = env.step(best_action)

    # Cetak informasi langkah
    print('--------------------------------------')
    print('Langkah ke  :', langkah)
    print('Best Action :', ACTION[best_action])
    print('New State   :', new_state)
    print('Reward      :', reward)
    print('Done        :', done)

    # Tampilkan visualisasi lingkungan
    env.render()
    current_state = new_state  # Perbarui state saat ini

    print()

--------------------------------------
Langkah ke  : 1
Best Action : BAWAH
New State   : 4
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 2
Best Action : BAWAH
New State   : 8
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 3
Best Action : KANAN
New State   : 9
Reward      : 0.0
Done        : False
  (Right)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 4
Best Action : BAWAH
New State   : 13
Reward      : 0.0
Done        : False
  (Down)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 5
Best Action : KANAN
New State   : 14
Reward      : 0.0
Done        : False
  (Right)
SFFF
FHFH
FFFH
HFFG

--------------------------------------
Langkah ke  : 6
Best Action : KANAN
New State   : 15
Reward      : 1.0
Done        : True
  (Right)
SFFF
FHFH
FFFH
HFFG



## **Periksa Apakah Agent Mampu Menyelesaikan Semua Peta**

Untuk setiap peta dalam daftar `peta`, program menampilkan peta yang sedang diperiksa. Kemudian, lingkungan permainan Frozen Lake dibuat berdasarkan deskripsi peta, dan agen dimulai di keadaan awal. Selanjutnya, agen melakukan langkah-langkah sampai langkah ke-6 atau sampai permainan selesai. Jika permainan selesai (variabel `done` bernilai `True`), artinya agen berhasil menyelesaikan peta, dan pesan "Status: Agent dapat menyelesaikan peta ini" dicetak. Jika tidak, pesan "Status: Agent tidak dapat menyelesaikan peta ini" dicetak. Proses ini diulang untuk setiap peta dalam daftar `peta`.

In [18]:
for peta_env in peta:  # Iterasi melalui setiap peta dalam daftar peta
    print("Peta   :", peta_env)  # Cetak deskripsi peta yang sedang diperiksa

    env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta_env)  # Buat lingkungan permainan Frozen Lake
    env.reset()  # Atur ulang lingkungan ke keadaan awal

    # Iterasi untuk agen melakukan langkah-langkah dalam lingkungan
    for langkah in range(1, 7):
        if langkah == 1:
            best_action = np.argmax(Q_table_all[index_peta][0])  # Ambil tindakan terbaik untuk langkah pertama
        else:
            best_action = np.argmax(Q_table_all[index_peta][current_state])  # Ambil tindakan terbaik berdasarkan state saat ini

        new_state, reward, done, info = env.step(best_action)  # Lakukan langkah terbaik dalam lingkungan

        current_state = new_state  # Perbarui state saat ini

    # Periksa apakah agen berhasil menyelesaikan permainan
    if done:
        print("Status : Agent dapat menyelesaikan peta ini")  # Cetak pesan jika agen berhasil menyelesaikan permainan
    else:
        print("Status : Agent tidak dapat menyelesaikan peta ini")  # Cetak pesan jika agen gagal menyelesaikan permainan

    print()  # Cetak baris kosong sebagai pemisah antara hasil dari setiap peta

Peta   : ['SFFF', 'FHFH', 'FFFH', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'FFHF', 'HFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SHFF', 'FHFH', 'FFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'HHFF', 'FFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFH', 'FFFH', 'HFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini



## **Menyimpan Model**

Perintah ini menggunakan modul pickle untuk menyimpan variabel Q_table_all ke dalam file dengan nama "Q_table_Frozen_Lake.model". Proses ini disebut serialisasi, yang mengonversi objek Python menjadi representasi byte yang dapat disimpan atau ditransmisikan, dan dapat diambil kembali (disebut deserialisasi) ke objek Python. Ini berguna ketika Anda ingin menyimpan variabel yang telah dihitung atau dilatih untuk digunakan nanti, tanpa perlu menjalankan kembali proses perhitungan atau pelatihan.

In [19]:
# Simpan tabel Q yang telah dilatih menggunakan modul pickle
pickle.dump(Q_table_all, open('Q_table_Frozen_Lake.model', 'wb'))

# Penjelasan penutup
print("Tabel Q untuk lingkungan Frozen Lake telah berhasil disimpan dalam file 'Q_table_Frozen_Lake.model'. Proses pelatihan agen dan penyimpanan tabel Q ini memungkinkan untuk digunakan kembali dalam pengujian atau aplikasi selanjutnya tanpa perlu melakukan pelatihan ulang.")

Tabel Q untuk lingkungan Frozen Lake telah berhasil disimpan dalam file 'Q_table_Frozen_Lake.model'. Proses pelatihan agen dan penyimpanan tabel Q ini memungkinkan untuk digunakan kembali dalam pengujian atau aplikasi selanjutnya tanpa perlu melakukan pelatihan ulang.


In [20]:
pip install pygame

In [23]:
import pygame
import gym
import numpy as np
import pickle
import time

# Muat Q_table yang telah dilatih sebelumnya
with open('Q_table_Frozen_Lake.model', 'rb') as f:
    Q_table_all = pickle.load(f)

# Daftar peta (sama seperti di kode sebelumnya)
peta = [
    ['SFFF','FHFH','FFFH','HFFG'],
    ['SFFF','FFHF','HFFF','HFFG'],
    ['SHFF','FHFH','FFFH','HHFG'],
    ['SFFF','HHFF','FFFF','HFFG'],
    ['SFFH','FFFH','HFFH','HHFG']
]

# Pilih peta yang ingin digunakan (misal peta index 0)
index_peta = 0
env = gym.make("FrozenLake-v0", is_slippery=False, desc=peta[index_peta])
env.reset()

# Konfigurasi tampilan grid
TILE_SIZE = 100       # Ukuran tiap petak (dalam piksel)
GRID_SIZE = 4         # Grid 4x4
WIDTH = TILE_SIZE * GRID_SIZE
HEIGHT = TILE_SIZE * GRID_SIZE

# Peta Frozen Lake berasal dari bytes, konversi ke string untuk kemudahan
# Warna tiap jenis petak:
# S : Start (hijau), F : Frozen (biru muda), H : Hole (coklat), G : Goal (emas)
COLORS = {
    'S': (50, 205, 50),    # hijau
    'F': (173, 216, 230),  # biru muda
    'H': (139, 69, 19),    # coklat
    'G': (255, 215, 0)     # emas
}

# Inisialisasi Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Frozen Lake - Reinforcement Learning Game")

def draw_grid(env, agent_state):
    # Konversi setiap elemen di env.desc menjadi string
    grid = [[tile.decode('utf-8') if isinstance(tile, bytes) else tile for tile in row] for row in env.desc]
    
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            tile = grid[i][j]
            color = COLORS[tile]
            rect = pygame.Rect(j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE)
            pygame.draw.rect(screen, color, rect)
            pygame.draw.rect(screen, (0, 0, 0), rect, 2)
    
    # Gambar agen sebagai lingkaran merah pada posisi state saat ini
    agent_pos = np.unravel_index(agent_state, (GRID_SIZE, GRID_SIZE))
    center_x = agent_pos[1] * TILE_SIZE + TILE_SIZE // 2
    center_y = agent_pos[0] * TILE_SIZE + TILE_SIZE // 2
    pygame.draw.circle(screen, (255, 0, 0), (center_x, center_y), TILE_SIZE // 4)

# Dapatkan Q_table untuk peta yang dipilih
Q_table = Q_table_all[index_peta]

# Variabel untuk loop game
running = True
agent_state = 0
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)

game_over = False
message = ""

# Loop utama simulasi
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not game_over:
        # Pilih aksi terbaik dari Q_table
        best_action = np.argmax(Q_table[agent_state])
        next_state, reward, done, info = env.step(best_action)
        agent_state = next_state

        # Cek kondisi terminal (selesai game)
        if done:
            game_over = True
            if reward == 1:
                message = "MENANG!"
            else:
                message = "KALAH!"
        
        # Gambar ulang grid dan posisi agen
        screen.fill((255, 255, 255))
        draw_grid(env, agent_state)
        pygame.display.flip()
        
        # Tambahkan delay untuk animasi (misalnya 0.5 detik per langkah)
        time.sleep(0.5)
    else:
        # Tampilkan pesan kemenangan/kekalahan di tengah layar
        screen.fill((255, 255, 255))
        draw_grid(env, agent_state)
        text = font.render(message, True, (0, 0, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)
        pygame.display.flip()
        
        # Tahan pesan selama 2 detik kemudian keluar
        time.sleep(2)
        running = False

    clock.tick(60)

pygame.quit()
